# Read-me

- For this project, we wanto to build a ensemble method from a combination of LightGBMClassifier, Random Forests and a Logistic Regressor. 
- To do this, firstly, we need to optimize our parameters. 
- We use bayesian optimization as it showed better than grid search (in relation to time).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
from sklearn.model_selection import train_test_split
from lib.utils import *

In [5]:
!pip install lightgbm
!pip install scikit-optimize

# Load data

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [20]:
X_train = np.load('data/input/X_train.npy')
y_train = np.load('data/input/y_train.npy')
X_test = np.load('data/input/X_test.npy')
y_test = np.load('data/input/y_test.npy')

# Bayesian Optimization

## LGBM

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [23]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
from skopt import forest_minimize
from lightgbm import LGBMClassifier

def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(X_train, y_train)
    
    p = mdl.predict_proba(X_valid)[:, 1]
    
    print(roc_auc_score(y_valid, p))
    
    return -average_precision_score(y_valid, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          ] 

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.9669884624440781
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2610
Function value obtained: -0.8507
Current minimum: -0.8507
Iteration No: 2 started. Evaluating function at random point.
[0.0010385556240017917, 2, 10, 0.14183771058242609, 0.7437489153990157, 249]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.9467859665646338
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0823
Function value obtained: -0.7306
Current minimum: -0.8507
Iteration No: 3 started. Evaluating function at random point.
[0.00209745522423282, 5, 6, 0.1541824778996655, 0.8682075103820793, 273]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be 

In [44]:
params = res.x

In [46]:
import pickle

pickle.dump(params,open('data/params/lgbm.pickle','wb'))

## RandomForestClassifier

In [47]:
from sklearn.ensemble import RandomForestClassifier
def tune_rf(params):
    print(params)
    
    bootstrap = params[0]
    max_depth = params[1]
    max_features = params[2]
    min_samples_leaf = params[3]
    min_samples_split = params[4]
    n_estimators = params[5]

    
    
    mdl = RandomForestClassifier(bootstrap=bootstrap, max_depth=max_depth, \
     max_features=max_features, min_samples_leaf=min_samples_leaf, \
        min_samples_split=min_samples_split, n_estimators=n_estimators,    \
                                 random_state=0, 
                         class_weight="balanced", n_jobs=-1)
    mdl.fit(X_train, y_train)
    
    p = mdl.predict_proba(X_valid)[:, 1]
    
    print(roc_auc_score(y_valid, p))
    
    return -average_precision_score(y_valid, p)


space = {'bootstrap': [True, False],
 'max_depth': [10, 20, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 4],
 'min_samples_split': [2, 10],
 'n_estimators': [200, 1000]}

space = list(space.values())
print(space)

res = forest_minimize(tune_rf, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

[[True, False], [10, 20, None], ['auto', 'sqrt'], [1, 4], [2, 10], [200, 1000]]
Iteration No: 1 started. Evaluating function at random point.
[True, 20, 'sqrt', 3, 9, 208]
0.9678832116788321
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.4848
Function value obtained: -0.8488
Current minimum: -0.8488
Iteration No: 2 started. Evaluating function at random point.
[True, 20, 'auto', 3, 7, 700]
0.9683659053449494
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.5672
Function value obtained: -0.8498
Current minimum: -0.8498
Iteration No: 3 started. Evaluating function at random point.
[False, None, 'sqrt', 4, 5, 883]
0.9661408052743112
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.2167
Function value obtained: -0.8358
Current minimum: -0.8498
Iteration No: 4 started. Evaluating function at random point.
[True, 20, 'sqrt', 3, 10, 592]
0.968283494231222
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.2496

In [48]:
params = res.x

In [49]:
import pickle

pickle.dump(params,open('data/params/rf.pickle','wb'))

## Logistic Regressor

In [50]:
from sklearn.linear_model import LogisticRegression
def tune_lr(params):
    print(params)
    
    C = params[0]
    solver = params[1]

    mdl = LogisticRegression(C=C, solver=solver,\
                         class_weight="balanced", n_jobs=-1)
    mdl.fit(X_train, y_train)
    
    p = mdl.predict_proba(X_valid)[:, 1]
    
    print(roc_auc_score(y_valid, p))
    
    return -average_precision_score(y_valid, p)


space = {'C': [1, 10],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
        }

space = list(space.values())
print(space)

res = forest_minimize(tune_lr, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

[[1, 10], ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']]
Iteration No: 1 started. Evaluating function at random point.
[5, 'lbfgs']
0.9375441488109254
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.7396
Function value obtained: -0.6452
Current minimum: -0.6452
Iteration No: 2 started. Evaluating function at random point.
[1, 'liblinear']
0.9368259948198729
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0517
Function value obtained: -0.6437
Current minimum: -0.6452
Iteration No: 3 started. Evaluating function at random point.
[8, 'sag']


/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1355: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn("'n_jobs' > 1 does not have any effect when"
/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.9315045914763362
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.1773
Function value obtained: -0.6171
Current minimum: -0.6452
Iteration No: 4 started. Evaluating function at random point.
[9, 'saga']
0.9284906993171651
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.1655
Function value obtained: -0.6018
Current minimum: -0.6452
Iteration No: 5 started. Evaluating function at random point.
[1, 'sag']


/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


0.9314574994113491
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1535
Function value obtained: -0.6164
Current minimum: -0.6452
Iteration No: 6 started. Evaluating function at random point.
[6, 'newton-cg']
0.9376030138921592
Iteration No: 6 ended. Evaluation done at random point.
Time taken: 0.4181
Function value obtained: -0.6454
Current minimum: -0.6454
Iteration No: 7 started. Evaluating function at random point.
[5, 'saga']
0.9285260183659053
Iteration No: 7 ended. Evaluation done at random point.
Time taken: 0.1957
Function value obtained: -0.6020
Current minimum: -0.6454
Iteration No: 8 started. Evaluating function at random point.
[4, 'saga']


...

Iteration No: 46 ended. Search finished for the next optimal point.
Time taken: 0.2829
Function value obtained: -0.6439
Current minimum: -0.6459
Iteration No: 47 started. Searching for the next optimal point.
[8, 'newton-cg']
0.9376736519896397
Iteration No: 47 ended. Search finished for the next optimal point.
Time taken: 0.3021
Function value obtained: -0.6456
Current minimum: -0.6459
Iteration No: 48 started. Searching for the next optimal point.
[7, 'newton-cg']
0.9376618789733929
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.2826
Function value obtained: -0.6455
Current minimum: -0.6459
Iteration No: 49 started. Searching for the next optimal point.
[10, 'newton-cg']
0.93770897103838


/home/kevintakano/miniconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.2922
Function value obtained: -0.6459
Current minimum: -0.6459
Iteration No: 50 started. Searching for the next optimal point.
[7, 'sag']
0.931516364492583


/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.4119
Function value obtained: -0.6175
Current minimum: -0.6459


In [51]:
params = res.x

import pickle

pickle.dump(params,open('data/params/lr.pickle','wb'))

/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/kevintakano/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l